In [ ]:
# | default_exp mkdocs

In [ ]:
# | export

from typing import *

import os
import re
import collections
from pathlib import Path
import textwrap
import shutil
import types
import pkgutil
import importlib
import subprocess # nosec: B404
import shlex
import sys
import multiprocessing

import typer
from typer.testing import CliRunner

from configupdater import ConfigUpdater, Section
from configupdater.option import Option

from configparser import ConfigParser
from fastcore.script import call_parse

import nbdev
from nbdev.serve import proc_nbs
from nbdev.process import NBProcessor
from nbdev.frontmatter import FrontmatterProc
from nbdev.quarto import prepare as nbdev_prepare
from fastcore.meta import delegates
from nbdev.quarto import refresh_quarto_yml, _nbglob_docs, _pre_docs, nbdev_readme, _sprun#, nbdev_docs
from nbdev.doclinks import nbdev_export
from fastcore.shutil import move



import nbconvert

from nbdev_mkdocs._package_data import get_root_data_path
from nbdev_mkdocs._helpers.cli_doc import generate_cli_doc

In [ ]:
import pytest
import numpy as np
from tempfile import TemporaryDirectory
import yaml

from nbdev.cli import nbdev_new

# Helpers

In [ ]:
# | export


def _get_value_from_config(root_path: str, config_name: str) -> str:
    """Get the value from settings.ini file"""

    settings_path = Path(root_path) / "settings.ini"
    config = ConfigParser()
    config.read(settings_path)
    if not config.has_option("DEFAULT", config_name):
        return ""
    return config["DEFAULT"][config_name]

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)
    ret_val = _get_value_from_config(d, "lib_path")
    print(ret_val)
    assert ret_val == "nbdev_mkdocs"

nbdev_mkdocs


## Create new

### Add requirements to settings

In [ ]:
# | export


def _add_requirements_to_settings(root_path: str):
    """Adds requirments needed for mkdocs to settings.ini

    Params:
        root_path: path to where the settings.ini file is located

    """
    _requirements_path = get_root_data_path() / "requirements.txt"
    with open(_requirements_path, "r") as f:
        _new_req_to_add = f.read()
        lines = _new_req_to_add.split("\n")
        lines = [s.strip() for s in lines]
        lines = [s for s in lines if s != ""]
        _new_req_to_add = " \\\n".join(lines)

    setting_path = Path(root_path) / "settings.ini"
    if not setting_path.exists():
        typer.secho(
            f"Path '{setting_path.resolve()}' does not exists! Please use --root_path option to set path to setting.ini file.",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    try:

        updater = ConfigUpdater()
        updater.read(setting_path)
    except Exception as e:
        typer.secho(
            f"Error while reading '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=2)

    try:
        if "requirements" not in updater["DEFAULT"]:
            updater["DEFAULT"].last_block.add_after.space(2).comment("### Optional ###").option("requirements", "")  # type: ignore

        old_req: str = updater["DEFAULT"]["requirements"].value  # type: ignore

        def remove_leading_spaces(s: str) -> str:
            return "\n".join([x.lstrip() for x in s.split("\n")])

        old_req = remove_leading_spaces(old_req)
        new_req = remove_leading_spaces(_new_req_to_add)
        if new_req in old_req:
            typer.secho(f"Requirements already added to '{setting_path.resolve()}'.")
            return

        req = old_req + " \\\n" + new_req
        req = textwrap.indent(req, " " * 4)

        req_option = Option(key="requirements", value=req)
        updater["DEFAULT"]["requirements"] = req_option
    except Exception as e:
        typer.secho(
            f"Error while updating requiremets in '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

    updater.update_file()

    typer.secho(f"Requirements added to '{setting_path.resolve()}'.")

    return

In [ ]:
with TemporaryDirectory() as d:
    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["requirements"] = Option(
        key="requirements", value="\\\n  nbdev>=2.3.7 \\\n  typer[all]==0.6.1"
    )
    updater.update_file()

    assert "mkdocs" not in updater["DEFAULT"]["requirements"].value

    # testing adding requirements
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    # do nothin if the requirements are already added
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    print(updater)

Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp69zyh95k/settings.ini'.
Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp69zyh95k/settings.ini'.
[DEFAULT]
# All sections below are required unless otherwise specified.
# See https://github.com/fastai/nbdev/blob/master/settings.ini for examples.

### Python library ###
repo = nbdev-mkdocs
lib_name = %(repo)s
version = 0.0.1rc2
min_python = 3.7
license = apache2

### nbdev ###
doc_path = _docs
lib_path = nbdev_mkdocs
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
black_formatting = True

### Docs ###
branch = main
custom_sidebar = False
doc_host = https://%(user)s.github.io
doc_baseurl = /%(repo)s
git_url = https://github.com/%(user)s/%(repo)s
title = %(lib_name)s

### PyPI ###
audience = Developers
author = airt
author_email = info@airt.ai
copyright = 2022 onwards, %(author)s
description = Extension of nbdev for generating documenta

### Create mkdocs dir

In [ ]:
# | export


def _create_mkdocs_dir(root_path: str):
    mkdocs_template_path = get_root_data_path() / "mkdocs_template"
    if not mkdocs_template_path.exists():
        typer.secho(
            f"Unexpected error: path {mkdocs_template_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)
    dst_path = Path(root_path) / "mkdocs"
    if dst_path.exists():
        typer.secho(
            f"Directory {dst_path.resolve()} already exist, skipping its creation.",
        )
    else:
        shutil.copytree(mkdocs_template_path, dst_path)
        #         shutil.move(dst_path.parent / "mkdocs_template", dst_path)
        typer.secho(
            f"Directory {dst_path.resolve()} created.",
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_dir(d)

    print("\n".join([str(p) for p in (Path(d) / "mkdocs").glob("**/*")]))

Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs created.
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/overrides
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/site
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/overrides/main.html
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs/images
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs/javascripts
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs/stylesheets
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs/images/favicon.ico
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs/javascripts/mathjax.js
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/mkdocs/docs/javascripts/extra.js
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2omxm7nk/

### Create Mkdocs.yml

In [ ]:
# | export

_mkdocs_template_path = get_root_data_path() / "mkdocs_template.yml"

In [ ]:
assert _mkdocs_template_path.exists()

In [ ]:
# | export

with open(_mkdocs_template_path, "r") as f:
    _mkdocs_template = f.read()

In [ ]:
print(_mkdocs_template)

# Site
site_name: {title}
site_url: {doc_host}{doc_baseurl}
site_author: {author}
site_description: {description}
  
# Repository
repo_name: {repo}
repo_url: {git_url}
edit_uri: ""

copyright: {copyright}

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
            - https://docs.python.org/3/objects.inv
        options:
            heading_level: 2
            show_category_heading: true
            show_root_heading: true
            show_root_toc_entry: true
            show_signature_annotations: true
            show_if_no_docstring: true
            
markdown_extensions:
    - pymdownx.arithmatex:
        generic: true
    - pymdownx.inlinehilite
    - pymdownx.details
    - pymdownx.emoji
    - pymdownx.magiclink
    - pymdownx.superfences
    - pymdownx.tasklist
    - pymdownx.highlight:
        linenums: false
    - pymdownx.snippets:
        check_paths: true
    - pymdownx.t

In [ ]:
# | export
def _get_kwargs_from_settings(
    settings_path: Path, mkdocs_template: Optional[str] = None
) -> Dict[str, str]:
    config = ConfigParser()
    config.read(settings_path)
    if not mkdocs_template:
        mkdocs_template = _mkdocs_template
    keys = [s[1:-1] for s in re.findall("\{.*?\}", _mkdocs_template)]
    kwargs = {k: config["DEFAULT"][k] for k in keys}
    return kwargs

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    kwargs = _get_kwargs_from_settings(settings_path)

    actual = _mkdocs_template.format(**kwargs)

kwargs

{'title': 'nbdev-mkdocs',
 'doc_host': 'https://airtai.github.io',
 'doc_baseurl': '/nbdev-mkdocs',
 'author': 'airt',
 'description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo': 'nbdev-mkdocs',
 'git_url': 'https://github.com/airtai/nbdev-mkdocs',
 'copyright': '2022 onwards, airt'}

In [ ]:
# | export


def _create_mkdocs_yaml(root_path: str):
    try:
        # create mkdocs folder if necessary
        mkdocs_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
        mkdocs_path.parent.mkdir(exist_ok=True)
        # mkdocs.yml already exists, just return
        if mkdocs_path.exists():
            typer.secho(
                f"Path '{mkdocs_path.resolve()}' exists, skipping generation of it."
            )
            return

        # get default values from settings.ini
        settings_path = Path(root_path) / "settings.ini"
        kwargs = _get_kwargs_from_settings(settings_path)
        mkdocs_yaml_str = _mkdocs_template.format(**kwargs)
        with open(mkdocs_path, "w") as f:
            f.write(mkdocs_yaml_str)
            typer.secho(f"File '{mkdocs_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{mkdocs_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)

    with open(Path(d) / "mkdocs/mkdocs.yml") as f:
        y = yaml.safe_load(f)

y

File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp06jre36o/mkdocs/mkdocs.yml' generated.


{'site_name': 'nbdev-mkdocs',
 'site_url': 'https://airtai.github.io/nbdev-mkdocs',
 'site_author': 'airt',
 'site_description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo_name': 'nbdev-mkdocs',
 'repo_url': 'https://github.com/airtai/nbdev-mkdocs',
 'edit_uri': '',
 'copyright': '2022 onwards, airt',
 'docs_dir': 'docs',
 'site_dir': 'site',
 'plugins': [{'literate-nav': {'nav_file': 'SUMMARY.md'}},
  'search',
  {'mkdocstrings': {'handlers': {'python': {'import': ['https://docs.python.org/3/objects.inv'],
      'options': {'heading_level': 2,
       'show_category_heading': True,
       'show_root_heading': True,
       'show_root_toc_entry': True,
       'show_signature_annotations': True,
       'show_if_no_docstring': True}}}}}],
 'markdown_extensions': [{'pymdownx.arithmatex': {'generic': True}},
  'pymdownx.inlinehilite',
  'pymdownx.details',
  'pymdownx.emoji',
  'pymdownx.magiclink',
  'pymdownx.superfences',
  'pymdo

### Create summary_template.txt

In [ ]:
# | export

_summary_template = """- [Home](index.md)
{guides}
{api}
{cli}
{changelog}
"""


def _create_summary_template(root_path: str):
    try:
        # create mkdocs folder if necessary
        summary_template_path = Path(root_path) / "mkdocs" / "summary_template.txt"
        summary_template_path.parent.mkdir(exist_ok=True)
        # summary_template_path.yml already exists, just return
        if summary_template_path.exists():
            typer.secho(
                f"Path '{summary_template_path.resolve()}' exists, skipping generation of it."
            )
            return

        # generated a new summary_template_path.yml file
        with open(summary_template_path, "w") as f:
            f.write(_summary_template)
            typer.secho(f"File '{summary_template_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{summary_template_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)
    _create_summary_template(d)

    guides = """- Guides
    - [Guide one](docs/guide_1.md)"""

    api = """- API
    - [numpy.array](api/numpy/array.md)"""

    cli = """- CLI
    - [my-cli](cli/my_cli.md)"""

    changelog = "- [Releases](CHANGELOG.md)"

    with open(Path(d) / "mkdocs/summary_template.txt") as f:
        summary_template = f.read()
        summary = summary_template.format(
            guides=guides, api=api, cli=cli, changelog=changelog
        )
#         y = yaml.safe_load(summary)

print(summary)

File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7wjs9p2h/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7wjs9p2h/mkdocs/summary_template.txt' generated.
- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)



In [ ]:
# | export


def _replace_ghp_deploy_action(root_path: str):
    """Replace the default gh-pages deploy action file with the custom action template file

    Args:
        root_path: Project's root path
    """

    src_path = get_root_data_path() / "ghp_deploy_action_template.yml"
    if not src_path.exists():
        typer.secho(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)

    workflows_path = Path(root_path) / ".github" / "workflows"
    workflows_path.mkdir(exist_ok=True, parents=True)

    dst_path = Path(workflows_path) / "deploy.yaml"
    shutil.copyfile(src_path, dst_path)

In [ ]:
with TemporaryDirectory() as d:
    assert not (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()
    _replace_ghp_deploy_action(d)
    assert (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()

### Bringing it all together

In [ ]:
# | export

from contextlib import contextmanager

@contextmanager
def set_cwd(cwd_path: Union[Path, str]):
    
    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)
    
    try:    
        nbdev.config.get_config.cache_clear()
        yield
    finally:
        os.chdir(original_cwd)

In [ ]:
with TemporaryDirectory() as d:
    with set_cwd(d):
        assert Path(os.getcwd()) == Path(d).resolve(), f"{os.getcwd()=}, {Path(d).resolve()=}"

In [ ]:
# | export


@delegates(_nbglob_docs)
def new(root_path: str, **kwargs):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.

    Params:
        root_path: path under which mkdocs directory will be created
    """ 
    _add_requirements_to_settings(root_path)
    _create_mkdocs_dir(root_path)
    _create_mkdocs_yaml(root_path)
    _create_summary_template(root_path)
    _replace_ghp_deploy_action(root_path)

@call_parse
def new_cli(root_path: str):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.
    """
    new(root_path)

In [ ]:
def run_nbdev_new(d):
    with set_cwd(d):
        nbdev_new(repo="repo" ,branch="branch" ,user="user", author="author", author_email="author@mail.com", description="description")
    

In [ ]:


with TemporaryDirectory() as d:
    run_nbdev_new(d)
    
    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    mkdocs_path = Path(d) / "mkdocs"
    
    assert mkdocs_path.exists()
    assert (mkdocs_path / "mkdocs.yml").exists()
    assert (mkdocs_path / "overrides" / "main.html").exists()
    assert (mkdocs_path / "site").exists()
    assert (mkdocs_path / "summary_template.txt").exists()

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiacrnkqz/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiacrnkqz/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiacrnkqz/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiacrnkqz/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md



## Build

### Build markdown files

In [ ]:
# | export


def _get_nbs_for_markdown_conversion(cache: Path):
    """Get a list of notebooks that needs to be converted to markdown.

    Args:
        cache: Path to the nbs cache folder
    """
    return list(cache.glob("index.ipynb")) + list(cache.glob("./guides/*.ipynb"))

In [ ]:
with TemporaryDirectory() as d:    
    
    run_nbdev_new(d)

    
    (Path(d)/"nbs"/"guides").mkdir()
    shutil.copyfile(Path(d) / "nbs" / "00_core.ipynb", Path(d) / "nbs" / "guides" / "guide_00_core.ipynb")
    assert (Path(d) / "nbs" / "guides" / "guide_00_core.ipynb").exists()
    
    new(d)
    
    nbs_path = Path(d) / "nbs"
    nbs = _get_nbs_for_markdown_conversion(nbs_path)
    
    nbs = [str(nb) for nb in nbs]
    print(nbs)
    assert f"{d}/nbs/guides/guide_00_core.ipynb" in nbs, f"{d}/nbs/guides/guide_00_core.ipynb"
    

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2k5sjiii/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2k5sjiii/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2k5sjiii/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2k5sjiii/mkdocs/summary_template.txt' generated.
['/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2k5sjiii/nbs/index.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2k5sjiii/nbs/guides/guide_00_core.ipynb']


Output created: _docs/README.md



In [ ]:
# | export


def _generate_markdown_from_nbs(root_path: str):
    
    doc_path = Path(root_path) / "mkdocs" / "docs"
    doc_path.mkdir(exist_ok=True, parents=True)
    
    with set_cwd(root_path):
           
        nbs_path = _get_value_from_config(root_path, "nbs_path")
        path = Path(root_path) / nbs_path        
        
        cache = proc_nbs()

        notebooks = _get_nbs_for_markdown_conversion(cache)

        for nb in notebooks:
            dir_prefix = str(nb.parent)[len(str(cache)) + 1 :]
            dst_md = doc_path / f"{dir_prefix}" / f"{nb.stem}.md"
            dst_md.parent.mkdir(parents=True, exist_ok=True)

            cmd = f"cd {cache} && quarto render {nb} -o {nb.stem}.md -t gfm --no-execute"
            _sprun(cmd)

            src_md = cache / "_docs" / f"{nb.stem}.md"
            shutil.move(src_md, dst_md)
   

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    
    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    (Path(d)/"nbs"/"guides").mkdir()
    shutil.copyfile(Path(d) / "nbs" / "00_core.ipynb", Path(d) / "nbs" / "guides" / "guide_00_core.ipynb")
    assert (Path(d) / "nbs" / "guides" / "guide_00_core.ipynb").exists()
    
    new(d)
    
    _generate_markdown_from_nbs(root_path=d)

    # check markdown files
    print("Checks:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert len(mds) > 0, len(mds)

settings.ini created.
Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjvz7q750/settings.ini'.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/README.md



Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjvz7q750/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjvz7q750/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjvz7q750/mkdocs/summary_template.txt' generated.


pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/index.md

pandoc -o ../guide_00_core.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_core.html
  standalone: true
  default-image-extension: png
  
metadata
  description: Fill in a module description here
  title: core
  
Output created: ../_docs/guide_00_core.md



Checks:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjvz7q750/mkdocs/docs/index.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjvz7q750/mkdocs/docs/guides/guide_00_core.md


In [ ]:
# | export


def _replace_all(text: str, dir_prefix: str) -> str:
    """Replace the images relative path in the markdown text

    Args:
        text: String to replace
        dir_prefix: Sub directory prefix to append to the image's relative path

    Returns:
        The text with the updated images relative path
    """
    _replace = {}
    _pattern = re.compile(r"!\[[^\]]*\]\(([^https?:\/\/].*?)\s*(\"(?:.*[^\"])\")?\s*\)")
    _matches = [match.groups()[0] for match in _pattern.finditer(text)]

    if len(_matches) > 0:
        for m in _matches:
            _replace[m] = (
                os.path.normpath(Path("../images/nbs/").joinpath(f"{dir_prefix}/{m}"))
                if len(dir_prefix) > 0
                else f"images/nbs/{m}"
            )

        for k, v in _replace.items():
            text = text.replace(k, v)

    return text

In [ ]:
text = """![Git Repo_Clone_Page](../img/test.png)
![Git Repo_Clone_Page](images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

expected = """![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

dir_prefix = "guides"
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected

text = """![Git Repo_Clone_Page](img/test.png)
![Git Repo_Clone_Page](guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

expected = """![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

dir_prefix = ""
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected

![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)

![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)



In [ ]:
# | export

def _update_path_in_markdown(cache: Path, doc_path: Path):
    """Update guide images relative path in the markdown files

    Args:
        cache: Path to the nbs cache directory
        doc_path: Path to the mkdocs/docs directory
    """
    notebooks = _get_nbs_for_markdown_conversion(cache)

    for nb in notebooks:
        dir_prefix = str(nb.parent)[len(str(cache)) + 1 :]
        md = doc_path / f"{dir_prefix}" / f"{nb.stem}.md"

        with open(Path(md), "r") as f:
            _new_text = f.read()
            _new_text = _replace_all(_new_text, dir_prefix)
        with open(Path(md), "w") as f:
            f.write(_new_text)


def _copy_guide_images_to_docs_dir(root_path: str):
    """Copy guide images to the docs directory

    Args:
        root_path: path under which mkdocs directory will be created
    """
    # Reference: https://github.com/quarto-dev/quarto-cli/blob/main/src/core/image.ts#L38
    image_extensions = [
        ".apng",
        ".avif",
        ".gif",
        ".jpg",
        ".jpeg",
        ".jfif",
        ".pjpeg",
        ".pjp",
        ".png",
        ".svg",
        ".webp",
    ]

    cache = proc_nbs()
    nbs_images_path = [
        p for p in Path(cache).glob(r"**/*") if p.suffix in image_extensions
    ]

    if len(nbs_images_path) > 0:
        doc_path = Path(root_path) / "mkdocs" / "docs"
        img_path = Path(doc_path) / "images" / "nbs"
        for src_path in nbs_images_path:
            dir_prefix = str(src_path.parent)[len(str(cache)) + 1 :]
            dst_path = Path(img_path) / f"{dir_prefix}"
            dst_path.mkdir(exist_ok=True, parents=True)
            shutil.copy(src_path, dst_path)

        _update_path_in_markdown(cache, doc_path)

In [ ]:
def copy_guides(src, dst):

    src = Path(src)
    dst = Path(dst)
    assert src.exists()
    src_guides = (src / "nbs" / "guides")
    src_guides_len = len(src_guides.parts)
    
    dst_guides = (dst / "nbs" / "guides")
    
    for ext in [".ipynb", ".png", ".jpeg", ".jpg"]:
        for src_f in src_guides.glob(f"**/*{ext}"):
            dst_parts = dst_guides.parts + src_f.parts[src_guides_len:]
            dst_f = Path(*dst_parts)
            dst_f.parent.mkdir(exist_ok=True, parents=True)
            
            print(f"{src_f=}, {dst_f=}")
            shutil.copyfile(src_f, dst_f)


In [ ]:
with TemporaryDirectory() as d:
    
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    
    new(d)

    _generate_markdown_from_nbs(root_path=d)
    
    
    _copy_guide_images_to_docs_dir(d)

    # check image files
    print("Checks for images:")
    imgs = list((Path(d) / "mkdocs" / "docs" / "images" / "nbs").glob("**/*.*"))
    print("\n".join([str(img) for img in imgs]))
    assert len(imgs) > 0, len(imgs)

    # check markdown files
    print("Checks for markdown:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "guides" / "Guide_01_End_To_End_Walkthrough.md").exists()

    with open((Path(d) / "mkdocs" / "docs" / "guides" / "Guide_01_End_To_End_Walkthrough.md"), "r") as f:
        contents = f.read()
        assert "![Empty Git Repo](../images/nbs/guides/images/empty_git_repo.png)" in contents
        assert "![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)" in contents
        assert "![](../images/nbs/guides/images/jupyter_home.png)" in contents
        
    (Path(d) /"mkdocs"/ "docs" / "images" /"nbs"/"guides"/"images"/"jupyter_home.png").exists()
    (Path(d) /"mkdocs"/ "docs" / "images" /"nbs"/"guides"/"images"/"empty_git_repo.png").exists()

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


src_f=Path('../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb')
src_f=Path('../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb')
src_f=Path('../nbs/guides/images/say_hello.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/nbs/guides/images/say_hello.png')
src_f=Path('../nbs/guides/images/git_repo_clone_page.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/nbs/guides/images/git_repo_clone_page.png')
src_f=Path('../nbs/guides/images/CLI_command.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/nbs/guides/images/CLI_command.png')
src_f=Path('../nbs/guides/images/foo_doc_string.png'), dst_f=Pa

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md



Checks for images:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/say_hello.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/git_repo_clone_page.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/CLI_command.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/foo_doc_string.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/guide_notebook.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/jupyter_home.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/hello_class.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3/mkdocs/docs/images/nbs/guides/images/guide_3.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpdj31_4x3

### Build summary for guides

In [ ]:
# | export


def _get_title_from_notebook(nb_name: str) -> str:
    cache = proc_nbs()
    nb_path = Path(cache) / "guides" / f"{nb_name}.ipynb"

    if not nb_path.exists():
        typer.secho(
            f"Unexpected error: path {nb_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    nbp = NBProcessor(nb_path, procs=FrontmatterProc)
    nbp.process()
    return nbp.nb.frontmatter_["title"]

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    
    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    
    new(d)

    _generate_markdown_from_nbs(d)

    mds = sorted(
        [md for md in Path(d).glob("**/*.md") if md.name.lower().startswith("guide")]
    )

    title = [_get_title_from_notebook(m.stem) for m in mds]
    print(title)
    assert title == ["End-To-End Walkthrough"]

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


src_f=Path('../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpj4ptx9x8/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb')
src_f=Path('../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpj4ptx9x8/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb')
src_f=Path('../nbs/guides/images/say_hello.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpj4ptx9x8/nbs/guides/images/say_hello.png')
src_f=Path('../nbs/guides/images/git_repo_clone_page.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpj4ptx9x8/nbs/guides/images/git_repo_clone_page.png')
src_f=Path('../nbs/guides/images/CLI_command.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpj4ptx9x8/nbs/guides/images/CLI_command.png')
src_f=Path('../nbs/guides/images/foo_doc_string.png'), dst_f=Pa

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md



['End-To-End Walkthrough']


In [ ]:
# | export


def _generate_summary_for_guides(root_path: str) -> str:
    doc_path = Path(root_path) / "mkdocs" / "docs"
    mds = sorted(
        [md for md in doc_path.glob("**/*.md") if md.name.lower().startswith("guide")]
    )

    i = len(doc_path.parts)
    if len(mds) > 0:
        return "- Guides\n    - " + "    - ".join(
            [
                f"[{_get_title_from_notebook(md.stem)}]({'/'.join(md.parts[i:])})\n"
                for md in mds
            ]
        )
    else:
        return ""

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    
    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    
    new(d)

    _generate_markdown_from_nbs(root_path=d)
    guides = _generate_summary_for_guides(root_path=d)

print(guides)
assert "End-To-End Walkthrough" in guides

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


src_f=Path('../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqn9yy7u3/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb')
src_f=Path('../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqn9yy7u3/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb')
src_f=Path('../nbs/guides/images/say_hello.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqn9yy7u3/nbs/guides/images/say_hello.png')
src_f=Path('../nbs/guides/images/git_repo_clone_page.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqn9yy7u3/nbs/guides/images/git_repo_clone_page.png')
src_f=Path('../nbs/guides/images/CLI_command.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqn9yy7u3/nbs/guides/images/CLI_command.png')
src_f=Path('../nbs/guides/images/foo_doc_string.png'), dst_f=Pa

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md



- Guides
    - [End-To-End Walkthrough](guides/Guide_01_End_To_End_Walkthrough.md)



### Build API

In [ ]:
# | export


def get_submodules(package_name: str) -> List[str]:
    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(package_name)
    submodules = [
        info.name
        for info in pkgutil.walk_packages(m.__path__, prefix=f"{package_name}.")
    ]
    submodules = [
        x
        for x in submodules
        if not any([name.startswith("_") for name in x.split(".")])
    ]
    return submodules

In [ ]:
submodules = get_submodules("mkdocs")
submodules

['mkdocs.commands',
 'mkdocs.commands.babel',
 'mkdocs.commands.build',
 'mkdocs.commands.gh_deploy',
 'mkdocs.commands.new',
 'mkdocs.commands.serve',
 'mkdocs.commands.setup',
 'mkdocs.config',
 'mkdocs.config.base',
 'mkdocs.config.config_options',
 'mkdocs.config.defaults',
 'mkdocs.contrib',
 'mkdocs.contrib.search',
 'mkdocs.contrib.search.search_index',
 'mkdocs.exceptions',
 'mkdocs.livereload',
 'mkdocs.localization',
 'mkdocs.plugins',
 'mkdocs.structure',
 'mkdocs.structure.files',
 'mkdocs.structure.nav',
 'mkdocs.structure.pages',
 'mkdocs.structure.toc',
 'mkdocs.tests',
 'mkdocs.tests.babel_cmd_tests',
 'mkdocs.tests.base',
 'mkdocs.tests.build_tests',
 'mkdocs.tests.cli_tests',
 'mkdocs.tests.config',
 'mkdocs.tests.config.base_tests',
 'mkdocs.tests.config.config_options_tests',
 'mkdocs.tests.config.config_tests',
 'mkdocs.tests.gh_deploy_tests',
 'mkdocs.tests.integration',
 'mkdocs.tests.livereload_tests',
 'mkdocs.tests.localization_tests',
 'mkdocs.tests.new_tests

In [ ]:
# | export


def generate_api_doc_for_submodule(root_path: str, submodule: str) -> str:
    subpath = "API/" + submodule.replace(".", "/") + ".md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "w") as f:
        f.write(f"::: {submodule}")
    subnames = submodule.split(".")
    if len(subnames) > 2:
        return " " * 4 * (len(subnames) - 2) + f"- [{subnames[-1]}]({subpath})"
    else:
        return f"- [{submodule}]({subpath})"


def generate_api_docs_for_module(root_path: str, module_name: str) -> str:
    submodules = get_submodules(module_name)
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "API", ignore_errors=True)
    submodule_summary = "\n".join(
        [
            generate_api_doc_for_submodule(root_path=root_path, submodule=x)
            for x in submodules
        ]
    )
    return "- API\n" + textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    api_summary = generate_api_docs_for_module(d, "nbdev")
    print(api_summary)

    # make sure all paths exist
    paths = re.findall("\(.*?\)", api_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path

- API
    - [nbdev.clean](API/nbdev/clean.md)
    - [nbdev.cli](API/nbdev/cli.md)
    - [nbdev.config](API/nbdev/config.md)
    - [nbdev.doclinks](API/nbdev/doclinks.md)
    - [nbdev.export](API/nbdev/export.md)
    - [nbdev.extract_attachments](API/nbdev/extract_attachments.md)
    - [nbdev.frontmatter](API/nbdev/frontmatter.md)
    - [nbdev.imports](API/nbdev/imports.md)
    - [nbdev.maker](API/nbdev/maker.md)
    - [nbdev.merge](API/nbdev/merge.md)
    - [nbdev.migrate](API/nbdev/migrate.md)
    - [nbdev.process](API/nbdev/process.md)
    - [nbdev.processors](API/nbdev/processors.md)
    - [nbdev.qmd](API/nbdev/qmd.md)
    - [nbdev.quarto](API/nbdev/quarto.md)
    - [nbdev.release](API/nbdev/release.md)
    - [nbdev.serve](API/nbdev/serve.md)
    - [nbdev.serve_drv](API/nbdev/serve_drv.md)
    - [nbdev.showdoc](API/nbdev/showdoc.md)
    - [nbdev.sync](API/nbdev/sync.md)
    - [nbdev.test](API/nbdev/test.md)


In [ ]:
# | export


def _restrict_line_length(s: str, width: int = 80) -> str:
    """Restrict the line length of the given string.

    Args:
        s: Docstring to fix the width
        width: The maximum allowed line length

    Returns:
        A new string in which each line is less than the specified width.
    """
    _s = ""

    for blocks in s.split("\n\n"):
        sub_block = blocks.split("\n  ")
        for line in sub_block:
            line = line.replace("\n", " ")
            line = "\n".join(textwrap.wrap(line, width=width, replace_whitespace=False))
            if len(sub_block) == 1:
                _s += line + "\n\n"
            else:
                _s += "\n" + line + "\n" if line.endswith(":") else " " + line + "\n"
    return _s

In [ ]:
s = """usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory in the **root_path** directory and populates it with
initial values. You should edit mkdocs.yml file to customize it if needed.

positional arguments:
  root_path

optional arguments:
  -h, --help  show this help message and exit show this help message and exit show this help message and exit
  -h, --help  show this help message and exit
  --port PORT
  --domain DOMAIN
"""

width = 60
doc = _restrict_line_length(s, width)

print(doc)
assert all([len(line) <= width for line in doc.splitlines() if line.strip() != ""])

usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory
in the **root_path** directory and populates it with initial
values. You should edit mkdocs.yml file to customize it if
needed.


positional arguments:
 root_path

optional arguments:
 -h, --help  show this help message and exit show this help
message and exit show this help message and exit
 -h, --help  show this help message and exit
 --port PORT
 --domain DOMAIN



In [ ]:
# | export


def generate_cli_doc_for_submodule(root_path: str, cmd: str) -> str:

    cli_app_name = cmd.split("=")[0]
    module_name = cmd.split("=")[1].split(":")[0]
    method_name = cmd.split("=")[1].split(":")[1]

    subpath = f"CLI/{cli_app_name}.md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)

    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(module_name)
    if isinstance(getattr(m, method_name), typer.Typer):
        app = typer.Typer()
        app.command()(generate_cli_doc)
        runner = CliRunner()
        result = runner.invoke(app, [module_name, cli_app_name])
        cli_doc = str(result.stdout)
    else:
        cmd = f"{cli_app_name} --help"
        print(f"Not a typer command. Documenting: cmd={cmd}")

        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        cli_doc = subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
            cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        ).stdout.decode("utf-8")

        cli_doc = _restrict_line_length(cli_doc)
        cli_doc = "\n```\n" + cli_doc + "\n```\n"

    with open(path, "w") as f:
        f.write(cli_doc)

    return f"- [{cli_app_name}]({subpath})"


def generate_cli_docs_for_module(root_path: str, module_name: str) -> str:
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "CLI", ignore_errors=True)
    console_scripts = _get_value_from_config(root_path, "console_scripts")

    if not console_scripts:
        return ""

    submodule_summary = "\n".join(
        [
            generate_cli_doc_for_submodule(root_path=root_path, cmd=cmd)
            for cmd in console_scripts.split("\n")
        ]
    )

    return "- CLI\n" + textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    cli_summary = generate_cli_docs_for_module(d, "nbdev_mkdocs")
    print(cli_summary)

    # make sure all paths and content exist
    paths = re.findall("\(.*?\)", cli_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path

Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp32vauw86/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp32vauw86/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp32vauw86/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp32vauw86/mkdocs/summary_template.txt' generated.
Not a typer command. Documenting: cmd=nbdev_mkdocs_new --help
Not a typer command. Documenting: cmd=nbdev_mkdocs_prepare --help
Not a typer command. Documenting: cmd=nbdev_mkdocs_preview --help
- CLI
    - [nbdev_mkdocs](CLI/nbdev_mkdocs.md)
    - [nbdev_mkdocs_new](CLI/nbdev_mkdocs_new.md)
    - [nbdev_mkdocs_prepare](CLI/nbdev_mkdocs_prepare.md)
    - [nbdev_mkdocs_preview](CLI/nbdev_mkdocs_preview.md)


In [ ]:
# | export


def _copy_change_log_if_exists(
    root_path: Union[Path, str], docs_path: Union[Path, str]
) -> str:
    changelog = ""
    source_change_log_path = Path(root_path) / "CHANGELOG.md"
    dst_change_log_path = Path(docs_path) / "CHANGELOG.md"
    if source_change_log_path.exists():
        shutil.copy(source_change_log_path, dst_change_log_path)
        changelog = "- [Releases](CHANGELOG.md)"
    return changelog

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == ""

    change_log_path = Path(d) / "CHANGELOG.md"
    with open(change_log_path, "w") as f:
        f.write("CHANGELOG")

    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == "- [Releases](CHANGELOG.md)"

Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfjg7knqy/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfjg7knqy/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfjg7knqy/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfjg7knqy/mkdocs/summary_template.txt' generated.
change_log=
change_log=- [Releases](CHANGELOG.md)


### Brining it all together

In [ ]:
# | export


def build_summary(
    root_path: str,
    module: str,
):
    # create docs_path if needed
    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True)

    # copy README.md as index.md
    shutil.copy(Path(root_path) / "README.md", docs_path / "index.md")

    # generate markdown files
    _generate_markdown_from_nbs(root_path)

    # copy guide images to docs dir and update path in generated markdown files
    _copy_guide_images_to_docs_dir(root_path)

    # generates guides
    guides = _generate_summary_for_guides(root_path)

    # generate API
    api = generate_api_docs_for_module(root_path, module)

    # generate CLI
    cli = generate_cli_docs_for_module(root_path, module)

    # copy CHANGELOG.md as CHANGELOG.md is exists
    changelog = _copy_change_log_if_exists(root_path, docs_path)

    # read summary template from file
    with open(Path(root_path) / "mkdocs" / "summary_template.txt") as f:
        summary_template = f.read()

    summary = summary_template.format(
        guides=guides, api=api, cli=cli, changelog=changelog
    )
    summary = "\n".join(
        [l for l in [l.rstrip() for l in summary.split("\n")] if l != ""]
    )

    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    new(d)

    build_summary(d, "nbdev_mkdocs")

    with open(Path(d) / "mkdocs/docs/SUMMARY.md") as f:
        summary = f.read()

    print(summary)
    assert "- [Home](index.md)" in summary
    assert "- [Releases](CHANGELOG.md)" in summary

Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpd2zgnd6_/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpd2zgnd6_/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpd2zgnd6_/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpd2zgnd6_/mkdocs/summary_template.txt' generated.
Not a typer command. Documenting: cmd=nbdev_mkdocs_new --help
Not a typer command. Documenting: cmd=nbdev_mkdocs_prepare --help
Not a typer command. Documenting: cmd=nbdev_mkdocs_preview --help
- [Home](index.md)
- API
    - [nbdev_mkdocs.docstring](API/nbdev_mkdocs/docstring.md)
    - [nbdev_mkdocs.mkdocs](API/nbdev_mkdocs/mkdocs.md)
- CLI
    - [nbdev_mkdocs](CLI/nbdev_mkdocs.md)
    - [nbdev_mkdocs_new](CLI/nbdev_mkdocs_new.md)
    - [nbdev_mkdocs_prepare](CLI/nbdev_mkdocs_prepare.md)
    - [nbdev_mkdocs_preview](CLI/nbdev_mkdocs_preview.md)
- [Releases](CHANGE

### Copy CNAME if needed

In [ ]:
# | export


def copy_cname_if_needed(root_path: str):
    cname_path = Path(root_path) / "CNAME"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "CNAME"
    if cname_path.exists():
        dst_path.parent.mkdir(exist_ok=True, parents=True)
        shutil.copyfile(cname_path, dst_path)
        typer.secho(
            f"File '{cname_path.resolve()}' copied to '{dst_path.resolve()}'.",
        )
    else:
        typer.secho(
            f"File '{cname_path.resolve()}' not found, skipping copying..",
        )

In [ ]:
for has_cname in [True, False]:
    with TemporaryDirectory() as d:
        settings_path = Path(d) / "settings.ini"
        for fname in ["settings.ini", "README.md"] + ["CNAME"] if has_cname else []:
            shutil.copyfile(Path("..") / fname, Path(d) / fname)

        copy_cname_if_needed(d)
        if has_cname:
            assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
        else:
            assert not (Path(d) / "mkdocs" / "docs" / "CNAME").exists()

File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2ucded67/CNAME' copied to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2ucded67/mkdocs/docs/CNAME'.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp8qer7t4u/CNAME' not found, skipping copying..


In [ ]:
# | export

@delegates(_nbglob_docs)
def prepare(root_path: str, no_test: bool = False, **kwargs):
    """Prepares mkdocs for serving

    Params:
        root_path: path under which mkdocs directory will be created
    """
    with set_cwd(root_path):
        if no_test:
            nbdev_export.__wrapped__()
            refresh_quarto_yml()
            nbdev_readme.__wrapped__(chk_time=True)
        else:
            nbdev_prepare.__wrapped__()


        n_workers = multiprocessing.cpu_count()
        nbs_path = _get_value_from_config(root_path, "nbs_path")

        cache,cfg,path = _pre_docs(n_workers=n_workers, **kwargs)


        # copy cname if it exists
        copy_cname_if_needed(root_path)

        # get lib name from settings.ini
        lib_name = _get_value_from_config(root_path, "lib_name")
        lib_path = _get_value_from_config(root_path, "lib_path")

        build_summary(root_path, lib_path)

        cmd = f"mkdocs build -f {root_path}/mkdocs/mkdocs.yml"
        _sprun(cmd)

@call_parse
def prepare_cli(root_path: str):
    """Prepares mkdocs for serving"""  
    prepare(root_path)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    
    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    
    new(d)

    prepare(d)

    #     !ll {d}/mkdocs/

    assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists
    assert (Path(d) / "mkdocs" / "docs" / "API").exists
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "CHANGELOG.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "guides").exists
#     !ls {d}/mkdocs/docs

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


src_f=Path('../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmph6hz2p__/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb')
src_f=Path('../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmph6hz2p__/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb')
src_f=Path('../nbs/guides/images/say_hello.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmph6hz2p__/nbs/guides/images/say_hello.png')
src_f=Path('../nbs/guides/images/git_repo_clone_page.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmph6hz2p__/nbs/guides/images/git_repo_clone_page.png')
src_f=Path('../nbs/guides/images/CLI_command.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmph6hz2p__/nbs/guides/images/CLI_command.png')
src_f=Path('../nbs/guides/images/foo_doc_string.png'), dst_f=Pa

Output created: _docs/README.md



Success.
-----nbdev_prepare finished-----
kwargs={}
-----_pre_docs finished-----
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmph6hz2p__/CNAME' not found, skipping copying..


pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md

/Users/harishm/miniforge3/envs/nbdev-mkdocs/lib/python3.9/site-packages/mkdocstrings/handlers/python/__init__.py:13: UserWarning: The 'python-legacy' extra of mkdocstrings will become mandatory in the next release. We have no way to detect if you already specify it, so if you do, please ignore this wa

## Preview

In [ ]:
# | export


def preview(root_path: str, port: Optional[int] = None):
    """Previes mkdocs documentation

    Params:
        root_path: path under which mkdocs directory will be created
        port: port to use
    """
    with set_cwd(root_path):
        prepare(root_path=root_path, no_test=True)

        cmd = f"mkdocs serve -f {root_path}/mkdocs/mkdocs.yml -a 0.0.0.0"
        if port:
            cmd = cmd + f":{port}"

        with subprocess.Popen(  # nosec B603:subprocess_without_shell_equals_true
            shlex.split(cmd),
            stdout=subprocess.PIPE,
            bufsize=1,
            text=True,
            universal_newlines=True,
        ) as p:
            for line in p.stdout:  # type: ignore
                print(line, end="")

        if p.returncode != 0:
            typer.secho(
                f"Command cmd='{cmd}' failed!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(6)


@call_parse
def preview_cli(root_path: str, port: Optional[int] = None):
    """Previes mkdocs documentation"""
    preview(root_path, port)

In [ ]:
# | notest

with TemporaryDirectory() as d:
    run_nbdev_new(d)
    
    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    
    new(d)

#     prepare(d)
#     !cd {d} && nbdev_export
#     !cd {d} && pip install -e '.[dev]'
#     new(d)
    
    
    assert (Path(d) / "mkdocs" / "docs" / "API").exists
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "CHANGELOG.md").exists
    #     !ls {d}/mkdocs/docs

    preview(d, port=4000)

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  


src_f=Path('../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbm9km6pr/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb')
src_f=Path('../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbm9km6pr/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb')
src_f=Path('../nbs/guides/images/say_hello.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbm9km6pr/nbs/guides/images/say_hello.png')
src_f=Path('../nbs/guides/images/git_repo_clone_page.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbm9km6pr/nbs/guides/images/git_repo_clone_page.png')
src_f=Path('../nbs/guides/images/CLI_command.png'), dst_f=Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbm9km6pr/nbs/guides/images/CLI_command.png')
src_f=Path('../nbs/guides/images/foo_doc_string.png'), dst_f=Pa

Output created: _docs/README.md



File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbm9km6pr/CNAME' not found, skipping copying..


pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  description: description
  title: repo
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md

/Users/harishm/miniforge3/envs/nbdev-mkdocs/lib/python3.9/site-packages/mkdocstrings/handlers/python/__init__.py:13: UserWarning: The 'python-legacy' extra of mkdocstrings will become mandatory in the next release. We have no way to detect if you already specify it, so if you do, please ignore this wa

KeyboardInterrupt: 